In [63]:
import requests
import json
import pandas as pd
import pickle
import fasttext
import neattext.functions as nfx

embeddings model

In [64]:
model = fasttext.load_model("fasttext_model_teds_wg.bin")

fetch a scanR publication

In [65]:
url="https://scanr.enseignementsup-recherche.gouv.fr/api/scanr-publications/_search?filter=externalIds.type:doi"

In [66]:
response = requests.get(url, verify=False)

C:\Users\haallat\AppData\Roaming\Python\Python311\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'scanr.enseignementsup-recherche.gouv.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [67]:
response

<Response [200]>

In [53]:
page=response.json()

In [54]:
page.get('hits').get('hits')[3].get('_source').get('title')

{'default': 'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions',
 'en': 'Identification of healthy elderly fallers and non-fallers by gait analysis under dual-task conditions'}

In [38]:
externalIds=page.get('hits').get('hits')[3].get('_source').get('externalIds')
externalIds

In [53]:
doi=[x for x in externalIds if x['type']=='doi'][0]['id']
doi

'10.1191/0269215506cr929oa'

In [68]:
doi='10.1016/j.jrurstud.2016.09.009'  #ex de publi tagué "verte"

In [ ]:
doi='10.1016/j.jrurstud.2016.09.009'  #ex de publi tagué "verte"

In [69]:
url_OA=f"https://api.openalex.org/works?filter=doi:{doi}"
response_OA = requests.get(url_OA)
data = response_OA.json()

In [70]:
topics=data.get('results')[0].get('topics')
topics_names=' '.join([topic.get('display_name') for topic in topics])
topics_names

'Drivers and Impacts of Tropical Deforestation Agricultural Innovation and Livelihood Diversification Global Analysis of Ecosystem Services and Land Use'

In [71]:
title=data.get('results')[0].get('title')
title

'Sustainable development policies and the spread of land-sharing practices – A statistical assessment in a frontier region of the Brazilian Amazon'

In [72]:
locations=data.get('results')[0].get('locations')
if (locations!=[]):
    for k in range(len(locations)):
        if pd.isna(locations[k].get('source'))==False:
            if locations[k].get('source').get('type')=='journal':
                locations_names=locations[k].get('source').get('display_name')
                locations_id=locations[k].get('source').get('issn_l') 
                break
print(locations_names,locations_id)

Journal of Rural Studies 0743-0167


In [73]:
test=f"{title} {topics_names} {locations_names} {locations_id}"

In [74]:
test

'Sustainable development policies and the spread of land-sharing practices – A statistical assessment in a frontier region of the Brazilian Amazon Drivers and Impacts of Tropical Deforestation Agricultural Innovation and Livelihood Diversification Global Analysis of Ecosystem Services and Land Use Journal of Rural Studies 0743-0167'

0 -> pas vert
<br />
1 -> vert

In [75]:
model.predict(nfx.remove_stopwords(test.lower()), k=-1)

(('__label__adaptation', '__label__mitigation', '__label__science'),
 array([0.79819679, 0.12941273, 0.02596736]))